In [51]:
import ast
from sklearn.preprocessing import LabelEncoder

file = open("example_data.csv", "r")
all_attributes = []
all_actions = set()

data_dataframe = []

for line in file:
  # print(line)
  parts = line.split(",")
  parts[1] = parts[1].replace(";", ",")
  all_actions.add(int(parts[0]))
  # print(parts[1])
  list_part = ast.literal_eval(parts[1])
  # print(list_part)
  for attrs in list_part:
    all_attributes.append(attrs)
  if len(list_part) == 0:
    continue
  data_dataframe.append([int(parts[0]), list_part])

unique_attributes = set(all_attributes)
# print(unique_attributes)
# Encode the actions and attributes
attribute_encoder = LabelEncoder().fit(list(unique_attributes))
# print(attribute_encoder.classes_)

action_encoder = LabelEncoder().fit(list(all_actions))
print(action_encoder.classes_)

[0 1 3 4 5]


In [1]:
pip install hmmlearn

0
1
2
3
4


In [84]:
#in this version all the encoded attributes are merged into a single encode
import ast
from sklearn.preprocessing import LabelEncoder
import pandas as pd

file = open("example_data.csv", "r")
all_attributes = []
all_actions = set()

data_dataframe = []

for line in file:
  # print(line)
  parts = line.split(",")
  parts[1] = parts[1].replace(";", ",")
  all_actions.add(int(parts[0]))
  # print(parts[1])
  list_part = ast.literal_eval(parts[1])
  # print(list_part)
  for attrs in list_part:
    all_attributes.append(attrs)
  if len(list_part) == 0:
    continue
  # data_dataframe.append([int(parts[0]), list_part])

# unique_attributes = set(all_attributes)
# print(all_attributes)
# print(unique_attributes)
# Encode the actions and attributes
attribute_encoder = LabelEncoder().fit(list(unique_attributes))

file = open("example_data.csv", "r")
for line in file:
  parts = line.split(",")
  parts[1] = parts[1].replace(";", ",")
  list_part = ast.literal_eval(parts[1])
  # print(list_part)
  if len(list_part) == 0:
    continue
  x = attribute_encoder.transform(list_part)
  # print(''.join(str(p) for p in sorted(x)))
  encoded_interaction = ''.join(str(p) for p in sorted(x))
  data_dataframe.append([int(parts[0]), encoded_interaction])
  # for attrs in list_part:
  #   all_attributes.append(attrs)

# print(attribute_encoder.classes_)
data = pd.DataFrame(data_dataframe, columns=['Action', 'Attribute'])

interaction_encoder = LabelEncoder().fit(data['Attribute'])
print(interaction_encoder.classes_)
action_encoder = LabelEncoder().fit(data['Action'])
print(action_encoder.classes_)

data['Encoded_Attribute'] = attribute_encoder.fit_transform(data['Attribute'])
# print(data.head(20))

['029' '07' '079' '09' '1' '19' '1910' '29' '2912' '3456' '34569' '7' '8'
 '89' '8913' '891314' '9' '910' '91011' '911' '91113' '912' '913' '914'
 '91415']
[0 1 3 4 5]


In [85]:
# 2 using the new encoded interactions
from hmmlearn import hmm
import numpy as np

# Prepare the sequences for HMM
# This example assumes each action transition marks a new sequence
# Further preprocessing might be needed based on the actual sequence definition

# Group by action to identify sequences (simple approach for demonstration)
groups = data.groupby('Action')

sequences = []
lengths = []
for name, group in groups:
    sequences.append(group['Encoded_Attribute'].values.reshape(-1, 1))
    lengths.append(len(group))

# Combine all sequences for training
X = np.concatenate(sequences)
lengths = np.array(lengths)

# Define a Multinomial HMM
# Number of components is the number of unique actions
model = hmm.MultinomialHMM(n_components=len(data['Action'].unique()), n_iter=100, random_state=42)

# Fit the model
model.fit(X, lengths=lengths)

https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340


MultinomialHMM(n_components=5, n_iter=100,
               n_trials=array([ 9,  9,  9,  9,  9, 10, 10, 10, 10, 11, 11,  1, 12, 12, 13, 14,  3,
        3,  7,  7, 16, 22, 19, 17, 17,  5,  4, 16, 23,  0,  8, 21, 20, 18,
        6,  5,  5, 24,  2, 15, 10]),
               random_state=RandomState(MT19937) at 0x7C18EF9C3640)

In [98]:
# 2 using the new encoded interactions
from hmmlearn import hmm
import numpy as np

# Prepare the sequences for HMM
# This example assumes each action transition marks a new sequence
# Further preprocessing might be needed based on the actual sequence definition

# Group by action to identify sequences (simple approach for demonstration)
split = int(len(data)* 0.8)
groups = data[:split].groupby('Action')

sequences = []
lengths = []
for name, group in groups:
  # print(group)
  sequences.append(group['Encoded_Attribute'].values.reshape(-1, 1))
  lengths.append(len(group))

# print(groups.head())
# Combine all sequences for training
X = np.concatenate(sequences)
lengths = np.array(lengths)

# Define a Multinomial HMM
# Number of components is the number of unique actions
model = hmm.MultinomialHMM(n_components=len(data['Action'].unique()), n_iter=100, random_state=42)

# Fit the model
model.fit(X, lengths=lengths)

https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340


MultinomialHMM(n_components=5, n_iter=100,
               n_trials=array([ 9,  9,  9,  9,  9, 10, 10, 10, 10, 11, 11,  1, 12, 12, 13, 14,  3,
        3,  7,  7, 16, 22, 19, 17,  0,  8, 21, 20, 18,  2, 15, 10]),
               random_state=RandomState(MT19937) at 0x7C18EF9C3E40)

In [106]:
test_data = data['Attribute'].values[split:]
# print(new_sequence)
for new_sequence in test_data:
  new_sequence_encoded = attribute_encoder.transform([new_sequence]).reshape(-1, 1)
  # Predict the state sequence
  # logprob, state_sequence = model.decode(new_sequence_encoded, algorithm="viterbi")
  print(new_sequence_encoded)
  hidden_states = model.predict(new_sequence_encoded)
  print(hidden_states)

[[17]]
[1]
[[6]]
[1]
[[5]]
[1]
[[5]]
[1]
[[4]]
[1]
[[5]]
[1]
[[16]]
[1]
[[23]]
[1]
[[24]]
[1]


In [86]:
# Example: Predicting the state sequence for a new sequence of attributes
new_sequence = ['3456']  # New sequence of attributes
new_sequence_encoded = attribute_encoder.transform(new_sequence).reshape(-1, 1)

# Predict the state sequence
logprob, state_sequence = model.decode(new_sequence_encoded, algorithm="viterbi")
print(state_sequence)
# The last state in `state_sequence` can be interpreted as the predicted next action

[1]


In [ ]:
from hmmlearn import hmm
import numpy as np
import pandas as pd

data = pd.DataFrame(data_dataframe, columns=['Action', 'Attribute'])
# print(data.head())

# Prepare the sequences and their lengths for HMM
sequences = [attribute_encoder.transform(seq) for seq in data['Attribute']]
actions = action_encoder.transform(data['Action'])

encoded_sequences = [attribute_encoder.transform(seq).reshape(-1, 1) for seq in data['Attribute']]
print(encoded_sequences)


In [53]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# print(len(test_data))

# Prepare the sequences and their lengths for HMM, separately for training and testing
train_sequences = [attribute_encoder.transform(seq) for seq in train_data['Attribute']]
test_sequences = [attribute_encoder.transform(seq) for seq in test_data['Attribute']]

train_lengths = [len(s) for s in train_sequences]
test_lengths = [len(s) for s in test_sequences]

print(train_lengths)
print(test_lengths)
train_X = np.concatenate([s.reshape(-1, 1) for s in train_sequences])
test_X = np.concatenate([s.reshape(-1, 1) for s in test_sequences])

# Define and train the HMM on the training set
model = hmm.CategoricalHMM(n_components=len(unique_attributes) + len(data['Action'].unique()), n_iter=100, random_state=42)
print(len(train_X))
model.fit(train_X, lengths=train_lengths)

# To predict the sequence of states (including the next action) for a new observation sequence:
# Example using the first sequence from the test set
logprob, states = model.decode(test_X[:test_lengths[0]].reshape(-1, 1), algorithm="viterbi")


[5, 1, 2, 3, 5, 1, 2, 2, 2, 2, 4, 2, 5, 1, 4, 3, 3, 4, 2, 2, 4, 1, 2, 3, 1, 2, 4, 2, 5, 1, 3, 1]
[3, 3, 5, 2, 4, 3, 2, 2, 2]
84


ValueError: transmat_ rows must sum to 1 (got row sums of [1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.])